In [12]:
import numpy as np
import pandas as pd
import scipy as sp
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, GRU, LSTM, TimeDistributed,SimpleRNN, Flatten, Dense, Input, InputLayer, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAvgPool2D, MaxPool2D
from tensorflow.keras.models import Sequential
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [17]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time=np.linspace(0,1,n_steps)
    series = 0.5 * np.sin((time-offsets1)*(freq1*10+10))
    series+= 0.2 * np.sin((time-offsets2)*(freq2*20+20))
    series+= 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
    
    return series[..., np.newaxis].astype(np.float32)

In [18]:
n_steps=50
series = generate_time_series(10000, n_steps +1)
xt, yt = series[:7000, :n_steps],series[:7000, -1]
xv, yv = series[7000:9000, :n_steps],series[7000:9000, -1]
xte, yte = series[9000:, :n_steps],series[9000:, -1]

In [19]:
yp = xv[:,-1]
#niave solution
np.mean(keras.losses.mean_squared_error(yv, yp))

0.020978257

In [20]:
#Niave dense
model = Sequential([
    Flatten(input_shape=[50,1]),
    Dense(1)
])

model.compile(loss = "mse", optimizer = "adam", metrics=["accuracy"])
history = model.fit(xt, yt, epochs = 20, validation_data = (xv, yv),verbose=0)
model.evaluate(xte, yte)

32/32 [==============================] - 0s 716us/step - loss: 0.0037 - accuracy: 0.0000e+00


[0.003676652442663908, 0.0]

In [21]:
model = Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss = "mse")
history = model.fit(xt, yt, epochs = 20, validation_data = (xv, yv),verbose=0)
model.evaluate(xte, yte)

32/32 [==============================] - 0s 3ms/step - loss: 0.0027


0.002689632121473551

In [22]:
n_steps=50
series = generate_time_series(10000, n_steps +10)
xt, yt = series[:7000, :n_steps],series[:7000, -10:,0]
xv, yv = series[7000:9000, :n_steps],series[7000:9000,  -10:,0]
xte, yte = series[9000:, :n_steps],series[9000:,  -10:,0]

model = Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss = "mse")
history = model.fit(xt, yt, epochs = 20, validation_data = (xv, yv),verbose=0)
model.evaluate(xte, yte)

32/32 [==============================] - 0s 2ms/step - loss: 0.0127


0.01271074265241623

In [23]:
n_steps=50
series = generate_time_series(10000, n_steps +10)
Y = np.empty((10000,n_steps,10))
for step in range(1,10+1):
    Y[:,:,step-1] = series[:,step:step+n_steps,0]
    
Yt  = Y[:7000]
Yv  = Y[7000:9000]
Yte = Y[9000:]

model = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20, return_sequences=True),
    TimeDistributed(Dense(10))
])

def last_time_step_mse(Y,Yp):
    return keras.metrics.mean_squared_error(Y[:,-1],Yp[:,-1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss = "mse", optimizer = optimizer, metrics = [last_time_step_mse])
history = model.fit(xt, Yt, epochs = 20, validation_data = (xv, Yv),verbose=0)
model.evaluate(xte, Yte)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
32/32 [==============================] - 0s 2ms/step - loss: 0.1458 - last_time_step_mse: 0.1425


[0.14582772552967072, 0.142517551779747]

In [24]:
model = Sequential([
    LSTM(20, return_sequences=True, input_shape=[None, 1]),
    LSTM(20, return_sequences=True),
    TimeDistributed(Dense(10))
])

model.compile(loss = "mse")
history = model.fit(xt, Yt, epochs = 20, validation_data = (xv, Yv),verbose=0)
model.evaluate(xte, Yte)

32/32 [==============================] - 0s 4ms/step - loss: 0.1458


0.1458210051059723

In [25]:
model = Sequential([
    Input((None,1)),
    Conv1D(filters=20, kernel_size=4, strides=2, padding ="valid"),
    GRU(20, return_sequences=True),
    GRU(20, return_sequences=True),
    TimeDistributed(Dense(10))
])

model.compile(loss = "mse")
history = model.fit(xt, Yt[:,3::2], epochs = 20, validation_data = (xv, Yv[:,3::2]),verbose=0)
model.evaluate(xte, Yte[:,3::2])

32/32 [==============================] - 0s 2ms/step - loss: 0.1460


0.14595060050487518

In [26]:
model = Sequential()
model.add(Input((None,1)))
for rate in (1,2,4,8) * 2:
    model.add(Conv1D(filters=20, kernel_size=2, padding ="causal", activation="relu",dilation_rate=rate))  
model.add(Conv1D(filters=10, kernel_size=1))

model.compile(loss = "mse", optimizer = "adam", metrics =[last_time_step_mse])
history = model.fit(xt, Yt, epochs = 20, validation_data = (xv, Yv),verbose=0)
model.evaluate(xte, Yte)

32/32 [==============================] - 0s 2ms/step - loss: 0.1458 - last_time_step_mse: 0.1424


[0.14582794904708862, 0.14241839945316315]

**Natural Language Processing with RNNs and Attention**

In [83]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [84]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])
max_id = len(tokenizer.word_index)

In [85]:
dataset_size = tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1
train_size = dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])


In [86]:
n_steps=100
window_length = n_steps +1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [87]:
dataset=dataset.flat_map(lambda window: window.batch(window_length))


In [88]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [89]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [82]:
print(dataset, max_id)
model = Sequential([
    GRU(128, return_sequences=True, input_shape=[None, max_id], dropout = 0.2, recurrent_dropout=0.2),
    GRU(128, return_sequences=True, dropout = 0.2, recurrent_dropout=0.2),
    TimeDistributed(Dense(max_id, activation ="softmax"))
])

<PrefetchDataset shapes: ((None, None, 39), (None, None)), types: (tf.float32, tf.int32)> 39


In [59]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [62]:
history = model.fit(dataset,epochs=20)

Epoch 1/20


UnboundLocalError: local variable 'logs' referenced before assignment